# Script to compute Markowitz portfolio aggregated by all cases

In [ ]:

from scripts_overtoruism_analysis.Markowitz_pipeline import compute_filters_and_messages_for_case_analysis_diffusione_3
from global_import import *
%matplotlib inline
project_tourism = dh.get_or_create_project("overtourism1")
endpoint_url="http://minio:9000"

s3 = boto3.resource('s3',
                    endpoint_url=endpoint_url)

bucket = s3.Bucket('datalake')

cities_gdf = gpd.read_file(os.path.join(os.getcwd(),"Data","mavfa-fbk_AIxPA_tourism-delivery_2025.08.22-zoning","fbk-aixpa-turismo.shp"))

print("Extract list of files from bucket...")
list_files_od, list_files_presenze, list_str_dates_yyyymm = extract_filenames_and_date_from_bucket(bucket)
date_in_file_2_skip = {'projects/tourism/_origin/vodafone-aixpa/od-mask_202407.parquet':"2024-08-08",
                    'projects/tourism/_origin/vodafone-aixpa/od-mask_202408.parquet':"2024-07-23"}
# NOTE: Extract the null day
print("Initialize null day OD-presenze...")
df_presenze_null_days = extract_presences_vodafone_from_bucket(s3 = s3,
                                                            list_files_presenze = list_files_presenze, 
                                                            i = 2)                                                                                                      # NOTE: download the file from the bucket
df_presenze_null_days = add_is_weekday_from_period_presenze_null_days(df = df_presenze_null_days, 
                                                                      period_col= str_period_id_presenze, 
                                                                      is_weekday_col= col_str_is_week)

# NOTE: Extract the stack of presences -> the overtouristic dataframe for presences
print("Stacking the presences data...")
stack_df_presenze_original = concat_presences(list_files_presences = list_files_presenze, 
                                    s3 = s3, 
                                    col_str_day_od = col_str_day_od, 
                                    col_period_id = str_period_id_presenze)

# NOTE: Add holiday column
stack_df_presenze_original = add_holiday_columun_df_presenze(stack_df_presenze = stack_df_presenze_original, 
                                                    col_str_day_od = col_str_day_od,
                                                    public_holidays = public_holidays,
                                                    col_str_is_week = col_str_is_week)

list_all_hours = list(stack_df_presenze_original[str_time_block_id_presenze].unique())
list_all_visitors = list(stack_df_presenze_original[str_visitor_class_id_presenze].unique())
list_all_countries = list(stack_df_presenze_original[str_country_presenze].unique())
list_all_weekdays = list(stack_df_presenze_original[col_str_is_week].unique())
print(f"List all hours: {list_all_hours}, list all visitors: {list_all_visitors}, list all countries: {list_all_countries}, list all weekdays: {list_all_weekdays}")
case2column_names_diffusione_3 = init_dict_case2column_names_diffusione_3(AGGREGATION_NAMES_2_COLUMNS_AGGREGATION_PRESENCES)

for case_pipeline in case2column_names_diffusione_3.keys():
    print(f"Initialize dict presences output for case_pipeline {case_pipeline}...")
    
    dict_presences_output, case_2iterable_values = initialize_dict_presences_output_diffusione_3(
        dict_case2column_names = case2column_names_diffusione_3,
        list_all_hours = list_all_hours,
        list_all_visitors = list_all_visitors,
        list_all_countries = list_all_countries,
        list_all_weekdays = list_all_weekdays,
        case_pipeline = case_pipeline,
    )

    # Extract the iterables for this case
    dict_iterable_name_2_iterable_values = case_2iterable_values[case_pipeline]
    
    # Unfold all possible combinations of present iterables
    iterable_names = list(dict_iterable_name_2_iterable_values.keys())
    iterable_values = [dict_iterable_name_2_iterable_values[k] for k in iterable_names]

    print(f"\tIterating over {iterable_names}")

    for combo in product(*iterable_values):
        combo_dict = dict(zip(iterable_names, combo))

        # Extract values (or defaults if not in this case)
        time = combo_dict.get("time", None)
        visitor = combo_dict.get("visitor", None)
        country = combo_dict.get("country", None)
        is_weekday = combo_dict.get("weekday", None)

        print(f"\tCombination: {combo_dict}")

        dict_presences_output = nested_set_dict_column_names_presences_analysis(
            case_2iterable_values = case_2iterable_values,
            dict_presences_output = dict_presences_output,
            time = time,
            visitor = visitor,
            country = country,
            is_weekday = is_weekday,
            case_pipeline = case_pipeline,
        )
        # NOTE: Get the name of the columns for presences
        col_aggregated_presences = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                                name_key ="column_presences_no_baseline",case_pipeline = case_pipeline)
        col_aggregated_presences_baseline = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                                name_key ="column_presences_baseline",case_pipeline = case_pipeline)
        # NOTE: Aggregate at the same level of aggregation null day and touristic presenze 
        stack_df_presenze_original_week_day_base = aggregate_presences_new_version(df = stack_df_presenze_original, 
                                                                    list_columns_groupby = case2column_names_diffusione_3[case_pipeline] + [col_str_day_od],
                                                                    str_col_trips_to_be_aggregated = "PRESENCES",
                                                                    str_col_name_aggregated = col_aggregated_presences,
                                                                    method_aggregation = "sum")

        df_presenze_null_days_week_day_base = aggregate_presences_new_version(df = df_presenze_null_days, 
                                                                list_columns_groupby = case2column_names_diffusione_3[case_pipeline],
                                                                str_col_trips_to_be_aggregated = "PRESENCES",
                                                                str_col_name_aggregated = col_aggregated_presences_baseline,
                                                                method_aggregation = "sum")
        # NOTE: Average over all days.
        df_presenze_null_days_week_day_base = df_presenze_null_days_week_day_base.with_columns((pl.col(col_aggregated_presences_baseline)/30).floor())


        is_covariance_standardized = True
        cities_gdf = gpd.read_file(os.path.join(os.getcwd(),"Data","mavfa-fbk_AIxPA_tourism-delivery_2025.08.22-zoning","fbk-aixpa-turismo.shp"))
        dict_case_2_tuple_filters, dict_case_message = compute_filters_and_messages_for_case_analysis_diffusione_3(
                                                                    dict_case2column_names = case2column_names_diffusione_3,
                                                                    time_of_interest = time,
                                                                    visitor_of_interest = visitor,
                                                                    country_of_interest = country,
                                                                    is_weekday = is_weekday,
                                                                )
        # NOTE: Filter by weekday / holiday -> Doing every hour since compute average takes away all the columns (it is logically inconsistent)
        stack_df_presenze_week_day = stack_df_presenze_original_week_day_base.filter(dict_case_2_tuple_filters[case_pipeline])    
        df_presenze_null_days_week_day = df_presenze_null_days_week_day_base.filter(dict_case_2_tuple_filters[case_pipeline])



        ########################################################
        ############### NULL DAY INITIALIZATION ################
        ########################################################

        print("Compute the total number of presences for each AREA_ID...")


        #############################################################
        ############ PREPROCESS RAW DATA TO MARKOWITZ ###############
        ############################################################# 
        # NOTE: Set the columns name for the analysis
        column_return = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                                name_key ="column_portfolio",case_pipeline = case_pipeline)
        col_expected_return = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                                name_key ="column_expected_return",case_pipeline = case_pipeline)
        col_std = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                name_key ="column_std",case_pipeline = case_pipeline)
        str_column_cov = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                        name_key ="column_cov",case_pipeline = case_pipeline)
        
        str_col_portfolio = nested_get_output_from_key_presences_analysis(case_2iterable_values = case_2iterable_values,dict_presences_output = dict_presences_output,
                                                                        name_key ="column_portfolio",case_pipeline = case_pipeline)
        # NOTE: Extract the list of days
        list_str_days = list(stack_df_presenze_original[col_str_day_od].unique())
        # NOTE: Compute the normalized covariance -> It is not for testing but chooses the return from the expectation
        stack_df_presenze = compute_starting_risk_column_from_stack_df(df_presenze_null_days = df_presenze_null_days_week_day,
                                                    stack_df_presenze = stack_df_presenze_week_day,
                                                    str_area_id_presenze = str_area_id_presenze,
                                                    col_total_presences_tour_no_hour = col_aggregated_presences,
                                                    col_total_presences_oct_no_hour =col_aggregated_presences_baseline,
                                                    col_return = column_return
                                                    )        
        # NOTE: Compute the expected return -> It is not for testing but chooses the return from the expectation
        df_mean = compute_expected_return_from_stack_df(stack_df_presenze = stack_df_presenze,
                                                        col_return = column_return,                                      # NOTE: This is expected i markowitz to be: col_tot_diff_oct
                                                        col_expected_return = col_expected_return,
                                                        str_area_id_presenze = str_area_id_presenze,
                                                        is_return_standardized = is_covariance_standardized,
                                                        col_std = col_std
                                                        )
        print(f"stack_df_presenze: {stack_df_presenze.head()}, df_mean: {df_mean.head()}, ")
        # NOTE: We define here the expected return: -> other approaches could be inserted here.
        expected_return = df_mean[col_expected_return].to_numpy()
        # NOTE: Standardize the return time series
        stack_df_presenze_mean_var = standardize_return_stack_df(stack_df_presenze = stack_df_presenze,
                                                                df_mean = df_mean,
                                                                col_return = column_return,
                                                                str_area_id_presenze = str_area_id_presenze,
                                                                is_standardize_return = is_covariance_standardized,
                                                                col_std = col_std)

        correlation_df = compute_correlation_matrix_df_from_time_series(stack_df_presenze_mean_var = stack_df_presenze_mean_var,
                                                                        str_area_id_presenze = str_area_id_presenze,
                                                                        col_str_day_od = col_str_day_od,
                                                                        col_return = column_return,
                                                                        str_column_cov = str_column_cov)    
        # NOTE: Extract area_to_index and index_to_area
        area_to_index, index_to_area = get_area_id_to_idx_mapping(cov_df = correlation_df, 
                                                                str_area_id_presenze = str_area_id_presenze)

        ##############################################################
        ####################### RMT Clean Matrix #####################
        ##############################################################

        # NOTE: Compute q = T/N
        q = from_areas_and_times_to_q(area_to_index = area_to_index,
                                    list_str_days = list_str_days)

        
        # NOTE: Transform covariance DataFrame into numpy matrix and create area mapping
        cov_matrix_numpy = from_df_correlation_to_numpy_matrix(cov_df = correlation_df, 
                                                            str_area_id_presenze = str_area_id_presenze, 
                                                            str_column_cov = str_column_cov, 
                                                            area_to_index = area_to_index)
        # NOTE: Clean the correlation matrix using RMT
        C_clean, eigvals_clean, eigvecs = rmt_clean_correlation_matrix(C = cov_matrix_numpy, 
                                                                        q = q,
                                                                        is_bulk_mean = True)

        
        # NOTE: Compute MP limits and mask of significant eigenvalues
        if is_covariance_standardized:
            sigma = None
        else:
            sigma = np.mean(df_mean[col_std].to_numpy())
        lambda_minus, lambda_plus, mask_eigvals = compute_MP_limits_and_mask(eigvals_clean, 
                                                                            q, 
                                                                            is_covariance_standardized= is_covariance_standardized, 
                                                                            sigma = sigma)        
                                                                            
    #        plot_pastur(eigvals_clean)

        ##############################################################
        #################### Markowitz procedure #####################
        ##############################################################
        # NOTE: Extract portfolio weights from significant eigenpairs
        portfolio_weights = extract_portfolio_from_eigenpairs(C_clean = C_clean, 
                                                                eigvals_clean = eigvals_clean, 
                                                                eigvecs = eigvecs, 
                                                                expected_return = expected_return, 
                                                                sum_w = 1,
                                                                is_normalize_portfolio=True)
        # NOTE: Map portfolio weights to cities_gdf and plot -> compute the portoflio 
        cities_gdf = map_portfolio_numpy_to_cities_gdf(cities_gdf = cities_gdf,
                                        portfolio_weights = portfolio_weights,
                                        index_to_area = index_to_area,
                                        str_area_id_presenze = str_area_id_presenze,
                                        str_col_portfolio = str_col_portfolio)
        cities_gdf = cities_gdf.merge(df_mean.to_pandas(),on=str_area_id_presenze)
        path_output_base = os.path.join(os.getcwd(),"Output")
        path_base_portfolio = path_output_base 
        for branch in dict_presences_output.keys():
            path_base_portfolio = os.path.join(path_base_portfolio,f"{branch}")
            os.makedirs(path_base_portfolio,exist_ok=True)

    #    fig,ax = plt.subplots(1,3, figsize = (10,10))
        fig,ax = plt.subplots(1,1, figsize = (10,10))
        plot_polygons_and_with_scalar_field(cities_gdf,str_col_portfolio,ax,fig,title = f"portfolio {case_pipeline}")        
    #    plot_polygons_and_with_scalar_field(cities_gdf,col_expected_return,ax[1],fig,title = "<oct - day> ")
    #    plot_polygons_and_with_scalar_field(cities_gdf,col_std,ax[2],fig,title = "standard deviation day")
        plt.show(fig)
        plt.savefig(os.path.join(path_base_portfolio,f"{case_pipeline}_portfolio_map.png"),dpi =200)
        plt.close(fig)
        fig_pst,ax_pst = plot_pastur(eigvals_clean, q)
        plt.savefig(os.path.join(path_base_portfolio,f"{case_pipeline}_pastur_distribution_eigenvalues.png"),dpi =200)
        # NOTE: Plot portfolio map
        path_save_portfolio = os.path.join(path_base_portfolio,f"portfolio_map_all_aggregated_{case_pipeline}.html")
        os.makedirs(path_base_portfolio,exist_ok=True)
        cities_gdf.to_file(os.path.join(path_base_portfolio,f"geodataframe_input_plots_markowitz_{case_pipeline}.geojson"))
        gc.collect()

# Single Trajectory -> understanding single evolution

In [ ]:
from typing import List
class Trajectory:
    def __init__(self, df: pl.DataFrame, id_col: str, time_col: str, list_feature_cols: List[str]):
        self.df = df
        self.id_col = id_col
        self.time_col = time_col
        self.list_feature_cols = list_feature_cols

    def __get__(self):
        return self
    
    def _get_ids(self):
        return self.df[self.id_col].unique().to_list()
    

    def get_trajectory(self, id_value):
        self.t = self.df.filter(pl.col(self.id_col) == id_value).sort(pl.col(self.time_col))[self.time_col].to_numpy()
        if len(self.list_feature_cols) == 1:
            self.x = self.df.filter(pl.col(self.id_col) == id_value).sort(pl.col(self.time_col))[self.list_feature_cols[0]].to_numpy()
        else:
            self.x = self.df.filter(pl.col(self.id_col) == id_value).sort(pl.col(self.time_col))[self.list_feature_cols].to_numpy()

        return self.t, self.x


fig,ax = plt.subplots(1,1, figsize = (10,10))
trajs = Trajectory(df = stack_df_presenze, id_col = str_area_id_presenze, time_col = col_str_day_od, list_feature_cols = [column_return])
list_ids = trajs._get_ids()
for id_value in list_ids:
    if len(trajs.list_feature_cols) == 1:
        ax.plot(trajs.get_trajectory(id_value = id_value)[0], trajs.get_trajectory(id_value = id_value)[1], label = id_value)
    else:
        ax.plot(trajs.get_trajectory(id_value = id_value)[0], trajs.get_trajectory(id_value = id_value)[1], label = id_value)
    ax.set_xticklabels(labels= trajs.get_trajectory(id_value = id_value)[0],rotation=90)

ax.legend()


# 

In [ ]:
import geopandas as gpd
from Plots import plot_polygons_and_with_scalar_field
import matplotlib.pyplot as plt
gdf = gpd.read_file("/home/aamaduzzi/aixpa-overtourism-backend/scripts_overtoruism_analysis/Output/Prefestivo/goedataframe_input_plots_markowitz_all_aggregated.geojson")
fig,ax = plt.subplots(1,1, figsize = (10,10))
gdf["people_to_move"] = gdf.apply(lambda row: row["expected_return"]*row["portfolio"]*100*24, axis=1)
plot_polygons_and_with_scalar_field(gdf,"people_to_move",ax,fig,title = f"People to move Prefestivo")        
plt.show(fig)


In [ ]:
gdf = gpd.read_file("/home/aamaduzzi/aixpa-overtourism-backend/scripts_overtoruism_analysis/Output/Festivo/goedataframe_input_plots_markowitz_all_aggregated.geojson")
fig,ax = plt.subplots(1,1, figsize = (10,10))
gdf["people_to_move"] = gdf.apply(lambda row: row["expected_return"]*row["portfolio"]*100*24, axis=1)
plot_polygons_and_with_scalar_field(gdf,"people_to_move",ax,fig,title = f"People to move Festivo")        
plt.show(fig)


In [ ]:
# compute fit of the expected return as exponential and plot it
import numpy as np
from scipy.optimize import curve_fit
def exponential_fit(x, a, b):
    return a * np.exp(b * x)
hist, bin_edges = np.histogram(gdf["expected_return"], bins=30, density=True)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
popt, pcov = curve_fit(exponential_fit, bin_centers, hist, p0=[1, 1])
x_fit = np.linspace(min(gdf["expected_return"]), max(gdf["expected_return"]), 100)
y_fit = exponential_fit(x_fit, *popt)
plt.figure(figsize=(10,6))
plt.hist(gdf["expected_return"], bins=30, density=True, alpha=0.6, label='Data Histogram')
plt.plot(x_fit, y_fit, 'r-', label=f'Exponential Fit ${popt[0]:.2f}e^{{{popt[1]:.2f}x}}$')
plt.xlabel('Expected Return')
plt.ylabel('Density')
plt.title('Histogram of Expected Return with Exponential Fit')
plt.legend()
plt.show()

In [ ]:
gdf = gpd.read_file("/home/aamaduzzi/aixpa-overtourism-backend/scripts_overtoruism_analysis/Output/Feriale/goedataframe_input_plots_markowitz_all_aggregated.geojson")
fig,ax = plt.subplots(1,1, figsize = (10,10))
gdf["people_to_move"] = gdf.apply(lambda row: row["expected_return"]*row["portfolio"]*100*24, axis=1)
plot_polygons_and_with_scalar_field(gdf,"people_to_move",ax,fig,title = f"People to move Feriale")        
plt.show(fig)


# RandomTree degree 5 to predict the line

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

error_rf = []
error_gb = []
fracion = 0.1
ns = np.arange(10, int(3010), 50)
for n in ns:
    t = np.arange(n)
    y = np.sin(0.02 * t) + 0.3 * np.random.randn(n)

    # Construct lag features
    p = int(fracion * n)
    X = np.column_stack([y[i:n - p + i] for i in range(p)])
    y_target = y[p:]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.2, shuffle=False)

    # --- Random Forest ---
    rf = RandomForestRegressor(n_estimators=200, max_depth=5, random_state=0)
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)

    # --- Gradient Boosting ---
    gb = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=3, random_state=0)
    gb.fit(X_train, y_train)
    y_pred_gb = gb.predict(X_test)
    error_rf.append(mean_squared_error(y_test, y_pred_rf))
    error_gb.append(mean_squared_error(y_test, y_pred_gb))
    # Compare errors
    print("RF RMSE:", mean_squared_error(y_test, y_pred_rf))
    print("GB RMSE:", mean_squared_error(y_test, y_pred_gb))

plt.plot(ns, error_rf, label='Random Forest', color='blue')
plt.plot(ns, error_gb, label='Gradient Boosting', color='orange')
plt.xlabel('Number of samples')
plt.ylabel('Mean Squared Error')
plt.title('Model Performance vs. Number of Samples')
plt.legend()
plt.show()


In [ ]:
y_rf = np.concatenate([y_train, y_pred_rf])
y_gb = np.concatenate([y_train, y_pred_gb])

plt.plot(t[p:], y_gb, label='Predicted GB', color='orange')
plt.plot(t, y, label='True',color = 'blue',alpha=0.1)
plt.plot(t[p:], y_rf, label='Predicted RF', color='green',alpha =0.3)
plt.legend()
plt.show()